### 24절기와 공휴일 정보를 API를 통해 가져와서 DB에 저장
- 관리자 계정으로 추가

In [13]:
import requests, json
from urllib.parse import quote
import pandas as pd
import db_sqlite.anniversary_dao as adao

In [14]:
key_file = 'static/key/govdataapikey.txt'
with open(key_file) as f:
    govdata_key = f.read()

- 공휴일

In [15]:
base_holiday_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear='
url = f'{base_holiday_url}2020&ServiceKey={quote(govdata_key)}'
response = requests.get(url)

In [16]:
response.status_code

200

In [17]:
json_ob = json.loads(response.text)
json_ob

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'},
  'body': {'items': {'item': [{'dateKind': '01',
      'dateName': '1월1일',
      'isHoliday': 'Y',
      'locdate': 20200101,
      'seq': 1},
     {'dateKind': '01',
      'dateName': '설날',
      'isHoliday': 'Y',
      'locdate': 20200124,
      'seq': 1},
     {'dateKind': '01',
      'dateName': '설날',
      'isHoliday': 'Y',
      'locdate': 20200125,
      'seq': 1},
     {'dateKind': '01',
      'dateName': '설날',
      'isHoliday': 'Y',
      'locdate': 20200126,
      'seq': 1},
     {'dateKind': '01',
      'dateName': '설날',
      'isHoliday': 'Y',
      'locdate': 20200127,
      'seq': 1},
     {'dateKind': '01',
      'dateName': '삼일절',
      'isHoliday': 'Y',
      'locdate': 20200301,
      'seq': 1},
     {'dateKind': '01',
      'dateName': '제21대 국회의원선거',
      'isHoliday': 'Y',
      'locdate': 20200415,
      'seq': 1},
     {'dateKind': '01',
      'dateName': '부처님오신날',
      'isHoliday': 'Y

In [18]:
holidays_data = json_ob["response"]["body"]["items"]["item"]
holidays_data[0]

{'dateKind': '01',
 'dateName': '1월1일',
 'isHoliday': 'Y',
 'locdate': 20200101,
 'seq': 1}

In [19]:
df = pd.DataFrame(holidays_data)
df

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20200101,1
1,01,설날,Y,20200124,1
2,01,설날,Y,20200125,1
3,01,설날,Y,20200126,1
4,01,설날,Y,20200127,1
5,01,삼일절,Y,20200301,1
6,01,제21대 국회의원선거,Y,20200415,1
7,01,부처님오신날,Y,20200430,1
8,01,어린이날,Y,20200505,1
9,01,현충일,Y,20200606,1


- 24절기

In [20]:
base_24_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/get24DivisionsInfo?_type=json&numOfRows=50&solYear='
url = f'{base_24_url}2020&ServiceKey={quote(govdata_key)}'
response = requests.get(url)
response.status_code

200

In [21]:
json_ob = json.loads(response.text)
twentyfours_data = json_ob["response"]["body"]["items"]["item"]
twentyfours_data[0]

{'dateKind': '03',
 'dateName': '소한',
 'isHoliday': 'N',
 'kst': '0630      ',
 'locdate': 20200106,
 'seq': 1,
 'sunLongitude': 285}

In [22]:
tf = pd.DataFrame(twentyfours_data)
tf

,dateKind,dateName,isHoliday,kst,locdate,seq,sunLongitude
0,03,소한,N,0630,20200106,1,285
1,03,대한,N,2355,20200120,1,300
2,03,입춘,N,1803,20200204,1,315
3,03,우수,N,1357,20200219,1,330
4,03,경칩,N,1157,20200305,1,345
5,03,춘분,N,0320,20200320,1,0
6,03,청명,N,1638,20200404,1,15
7,03,곡우,N,2345,20200419,2,30
8,03,입하,N,0951,20200505,2,45
9,03,소만,N,2249,20200520,1,60


- DB 저장

In [23]:
holidays = []
for i in df.index:
    holidays.append((df.dateName[i], df.locdate[i].astype('str'), 1))
len(holidays)

18

In [27]:
adao.insert_anniv_many(holidays)

OperationalError: no such table: anniversary

In [25]:
twentyfours = []
for i in tf.index:
    twentyfours.append((tf.dateName[i], str(tf.locdate[i]), 0))
adao.insert_anniv_many(twentyfours)

In [26]:
for year in range(2020, 2025):
    for k, base_url in enumerate([base_holiday_url, base_24_url]):
        url = f'{base_url}{year}&ServiceKey={quote(govdata_key)}'
        response = requests.get(url)
        json_ob = json.loads(response.text)
        anniv_data = json_ob["response"]["body"]["items"]["item"]
        df = pd.DataFrame(anniv_data)

        anniv_list = []
        for i in df.index:
            flag = 1 if k == 0 else 0
            anniv_list.append((df.dateName[i], str(df.locdate[i]), flag, 'admin'))
        adao.insert_anniv_many(anniv_list)

OperationalError: no such table: anniversary